<a href="https://colab.research.google.com/github/CNordqui/Coursera_Capstone/blob/master/Coursera_Capstone_CNordqui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook is for the Coursera Capstone Project for the Applied Data Science Specialization offered by IBM.** 




**Introduction/Business Problem:**

Due to habitat destruction in Australia from human activity and wildfires, many animals are restricted to sanctuaries and conservation territories. For individuals living in Australia and tourists, they may want to see the amazing creatures of Australia that have recently had diminishing populations. However, it is difficult to find these animals or even determine in which province they can be found. Therefore, the solution is generating a map where individuals can find Herbivorous Marsupials (undoubtedly the most popular subset of Australian animals which includes kangaroos and koalas) in the various provinces of Australia. Tourists and adventurers will be able to use this map to determine which province would be best to travel to in order to see these amazing wonders of Australia. In order to generate the map, I will be using Australian data which includes the regions from which the animals can be found as well as Foursquare data to show where conservancies are in the region that has the most Herbivorous Marsupials. 

**Data:**

The Data for this project is from Foursquare as well as from the Atlas of Living Australia. The data was downloaded and then copy and pasted into an excel spreadsheet. I deleted the columns titled "Species" and "Invasive" because the species column had urls to learn more about the spcies and the invasive column only denoted one animal as being an invasive specie. Then, I deleted the column "Scientific Name Authorship" because this information just gave the credit to whoever first found the species and named it-data that won't be important in this project. I then uploaded the excel spreadsheet as seen below and cleaned it up so only rows that had all of the columns filled were displayed. I will be using this data to determine in which province on Australia someone can find the most Herbivorous Marsupials and will then be using Foursquare data to determine where most conservancies and sanctuaries in that province can be found. This data will solve the problems of where tourists and explorers should go in order to see these amazing animals. 

In [2]:
import pandas as pd
import numpy as np
#I copied the data from Wikipedia and then pasted it into an excel spreadsheet. And read it into the pandas dataframe here. 
df = pd.read_excel('Herbivorous Marsupials.xlsx')
#Cleaning up the data
df.dropna(subset=['Conservation'], inplace=True)
df.head()

,Species Name,Taxon Rank,Kingdom,Phylum,Class,Order,Family,Genus,Vernacular Name,Conservation
0,Phascolarctos cinereus,species,ANIMALIA,CHORDATA,MAMMALIA,DIPROTODONTIA,PHASCOLARCTIDAE,Phascolarctos,Koala,Queensland
1,Trichosurus vulpecula,species,ANIMALIA,CHORDATA,MAMMALIA,DIPROTODONTIA,PHALANGERIDAE,Trichosurus,Common Brushtail Possum,"South Australia, Queensland, Northern Territory"
2,Petaurus norfolcensis,species,ANIMALIA,CHORDATA,MAMMALIA,DIPROTODONTIA,PETAURIDAE,Petaurus,Squirrel Glider,"South Australia, Queensland, Victoria"
3,Pseudocheirus peregrinus,species,ANIMALIA,CHORDATA,MAMMALIA,DIPROTODONTIA,PSEUDOCHEIRIDAE,Pseudocheirus,Common Ringtail Possum,Queensland
4,Wallabia bicolor,species,ANIMALIA,CHORDATA,MAMMALIA,DIPROTODONTIA,MACROPODIDAE,Wallabia,Swamp Wallaby,"South Australia, Queensland"


In order to determine the province in Australia with the largest number of these Herbivorous Marsupials, I need to look at the column titled 'Conservation' that lists the provinces in which the marsupials are considered to be under conservation. By tallying up all the numbers, there are 16 considered conserved in Queensland, 2 in Victoria, 8 in South Australia, 3 in Northern Territoriy, and 2 in New South Wales. Therefore, the province that has the greatest number of Herbivorous Marsupials in Queensland. Therefore, we will be using Foursquare data to look up conservancies and sanctuaries for these animals in Queensland. See dataframe showing the locations of these animals below. 

In [3]:
df['Conservation']

0                                           Queensland
1     South Australia, Queensland, Northern Territory 
2                South Australia, Queensland, Victoria
3                                          Queensland 
4                          South Australia, Queensland
5                         South Australia, Queensland 
6                         South Australia, Queensland 
7      South Australia, Queensland, Northern Territory
8                                           Queensland
9                         South Australia, Queensland 
11                                         Queensland 
12                                         Queensland 
13              Queensland, Victoria, New South Wales 
14                        Queensland, New South Wales 
15                                         Queensland 
17                                    South Australia 
18                     Queensland, Northern Territory 
Name: Conservation, dtype: object

I did some research in the Queensland area since the region is much too big to do a Foursquare geodata analysis on, and discovered that the Gold Coast is home to the greatest number of zoos, wildlife rehabilitation centers, and wildlife sanctuaries, giving the best chance of the presence of herbivorous marsupials. Therefore, moving forward, I will be analyzing the region of the Gold Coast to prove with the Foursquare data that this region has the greatest herbivorous marsupial populations. I used Google Maps and data from Google and discovered there were many parks, national parks, and animal sanctuaries in the Gold Coast area. Therefore, I found three locations that seemed to be in a hot spot of sorts for these types of areas (Currumbin Wildlife Sanctuary, Tamborine National Park, and Springbrook National Park). Around each of the three areas I used markers to show where the nearest venues were, and displayed the dataframes so it was clear that there were many parks and natural areas where large herbivorous marsupial populations would be found. 

Below is the first analysis of the currumbin Wildlife Sanctuary data displayed after a map of the overall view of the Gold Coast in Queensland. 

In [4]:
CLIENT_ID = 'UJ0BSIUED1BG3O5GYR2ZH1JPI453E2JZYLFAGTEVWVGDHQKD' # your Foursquare ID
CLIENT_SECRET = 'IVZFIFBPEPDO0JHBX2K1NAZ2IX4ZFUUWNCS5OQSRQKLSA0AD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100
radius=500
# create map of Gold Coast, Australia using latitude and longitude values and by inputting an address that helps to make the data more specific 
#it is the address of Currumbin Wildlife Sanctuary
!pip install geopandas
!pip install geopy
from geopy.geocoders import Nominatim
address = '28 Tomewin St, Currumbin QLD 4223, Australia'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)

#creating and displaying the map of Gold Coast, Australia
import folium
latitude=location.latitude
longitude=location.longitude
map_Queensland = folium.Map(location= [latitude, longitude], zoom_start=8)
#get the url from foursquare in order to use foursquare to analyze the venues near this location
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
map_Queensland

     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 10.9MB 14.6MB/s 
     |████████████████████████████████| 14.7MB 305kB/s 


In [0]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
results = requests.get(url).json()

In [0]:
#creating a function that allows me to see all the different categories and the names of the different venues from the foursquare data
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
#Finding the nearby venues from the results
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
#displaying the nearby venues to the sanctuary and other venues on the Gold Coast in Queensland, Australia
nearby_venues

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Free Flight Bird Show,Zoo Exhibit,-28.139545,153.483614
1,Currumbin Wildlife Sanctuary,Zoo,-28.135910,153.488417
2,Honeyworld,Health Food Store,-28.135470,153.487520
3,Wild Burger,Burger Joint,-28.137708,153.485487
4,Koala Junction Kiosk,Snack Place,-28.137249,153.485227
5,australiana sheep shearing show,Park,-28.139574,153.483564
6,Binky Bill and Friends,National Park,-28.136017,153.488568
7,Currumbin Sanctuary Markets,Market,-28.135620,153.488759
8,Currumbin Wildlife Sanctuary Southbound Bus Stop,Bus Stop,-28.135300,153.489520


As you can see, there are several parks and zoos in the area, demonstrating that there are high animal populations as well as many recovery organizations geered towards rehabilitating herbivorous marsupials (with kangaroos being the most popular). Therefore, the Foursquare data confirmed that the Gold Coast of Queenland in Australia is the best place for tourists and locals of Australia to go in order to have a greater chance of seeing herbivorous marsupials. 

In [8]:
# add markers to map
map_Currubim = folium.Map(location= [latitude, longitude], zoom_start=15)
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Currubim)  
    
map_Currubim

Above is the map to see where all the animal sactuaries, zoos, etc. can be found around the currumbin Wildlife Sanctuary area. These venues are in the very heart of the Gold Coast, and therefore, represent the region at large well. Below is the analysis from Tamborine National Park, a bit NW of the Currumbin Wildlife Sanctuary. 

In [9]:
from geopy.geocoders import Nominatim
address = 'Cedar Creek QLD 4207, Australia'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)

#creating and displaying the map of Gold Coast, Australia
import folium
latitude=location.latitude
longitude=location.longitude
map_park = folium.Map(location= [latitude, longitude], zoom_start=12)
radius=2000
#get the url from foursquare in order to use foursquare to analyze the venues near this location
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
map_park

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
results = requests.get(url).json()
#Finding the nearby venues from the results
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
#displaying the nearby venues to the sanctuary and other venues on the Gold Coast in Queensland, Australia
nearby_venues


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Cedar creek park lands,Park,-27.846997,153.191916
1,Pho Ever Taste,Vietnamese Restaurant,-27.844237,153.180099
2,Kym Simons Earthmover,Construction & Landscaping,-27.845695,153.173950
3,Cedar Creek Park,Park,-27.838936,153.173548
4,KWR Roofing,Business Service,-27.829133,153.185838


The data shows that nearby venues include a few parks as well as venues that are not natural in essence. This is because I increased the radius to 2000m instead of 500m because the national park that served as the address was so large (and therefore, has high herbivorous marsupial populations due to its size and location). 

In [10]:
# add markers to map
map_park = folium.Map(location= [latitude, longitude], zoom_start=15)
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='pink',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_park)  
    
map_park

Above is the geospatial data of Tamborine National Park and shows the vastness of the area, as well as how close the rest of society is to the national park. I'm sure native animal populaitons and humans come into contact more often that what is desirous. Below is the data analysis from Springbrook National Park, more South to Tamborine National Park. 

In [11]:
from geopy.geocoders import Nominatim
address = 'Old School Rd, Springbrook QLD 4213, Australia'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)

#creating and displaying the map of Gold Coast, Australia
import folium
latitude=location.latitude
longitude=location.longitude
map_trail = folium.Map(location= [latitude, longitude], zoom_start=12)
radius=1500
#get the url from foursquare in order to use foursquare to analyze the venues near this location
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
map_park

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
results = requests.get(url).json()
#Finding the nearby venues from the results
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
#displaying the nearby venues to the sanctuary and other venues on the Gold Coast in Queensland, Australia
nearby_venues

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Canyon Lookout,Scenic Lookout,-28.220682,153.272928
1,Black fella falls,Trail,-28.214168,153.272095
2,Glow Worm Research Centre,Tourist Information Center,-28.201076,153.271940
3,Warrie Circuit,Trail,-28.219903,153.272679
4,Rosella's Cafe & Accommodation,Café,-28.220711,153.272524


By looking at the dataframe above, since I increased the radius to 1500m instead of 500m, it is clear that all around this area are trails, nature preserves, and cliffs. The radius helps to prove that this region is densely forested and is the perfect protected spot for any herbivorous marsupial populations around. 

In [12]:
# add markers to map
map_trail = folium.Map(location= [latitude, longitude], zoom_start=13)
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trail)  
    
map_trail

The geospatial data above shows the very large amount of national parks, parks, nature reserves, and protected land around the Springbrook National Park. This further shows how much protected land the Gold Coast in Queensland has and further proves that tourists and those native to Australia should come to the Gold Coast if they desire to see herbivorous marsupial populations. This spot is the best chance for seeing them because these animals are able to live in harmony with nature and remain largely uninterrupted by man in these protected regions. 